## Testing Modelling

In [1]:
%pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

data_dir = os.getenv('DATA_DIR')
model_dir = os.getenv('MODEL_DIR')

In [3]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(32)

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SimpleModule, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, padding=1, stride=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1, stride=1)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        return x

class DrowningDetectionCNN(nn.Module):
    def __init__(self, img_size):
        super(DrowningDetectionCNN, self).__init__()

        self.block1 = SimpleModule(3, 16)
        self.block2 = SimpleModule(16, 32)
        self.block3 = SimpleModule(32, 64)

        self.dropout = nn.Dropout(0.5)

        self.flatten = nn.Flatten()
        self.fc1 = nn.LazyLinear(3)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)

        x = self.dropout(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        return x

In [5]:
# dataset_loader.py
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def get_data_loaders(batch_size=32):
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    train_dataset = datasets.ImageFolder(r'C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\data\torch_data\cropped\train', transform=transform)
    val_dataset = datasets.ImageFolder(r'C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\data\torch_data\cropped\val', transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader

In [6]:
# train.py
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DrowningDetectionCNN(128).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_loader, val_loader = get_data_loaders()

def train(num_epochs=1):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")
        validate()

def validate():
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    f1_weighted = f1_score(all_labels, all_preds, average='weighted')
    acc_weighted = balanced_accuracy_score(all_labels, all_preds)
    print(f"Validation Accuracy: {acc:.2f}")
    print(f"Validation F1 (weighted): {f1_weighted:.2f}")
    print(f"Validation Balanced Accuracy: {acc_weighted:.2f}")

In [7]:
train()

Epoch 1/1, Loss: 0.7481298763304949
Validation Accuracy: 0.72
Validation F1 (weighted): 0.65
Validation Balanced Accuracy: 0.61


In [8]:
validate()

Validation Accuracy: 0.72
Validation F1 (weighted): 0.65
Validation Balanced Accuracy: 0.61


In [9]:
%pip install tensorflow==2.16.1

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.2.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.13.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.3.2-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-4.25.6-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.17.2-cp311-cp311-win_amd64.whl.metadata (

  You can safely remove it manually.
  You can safely remove it manually.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

import pandas as pd
import os
from dotenv import load_dotenv

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("CPU Available:", tf.config.list_physical_devices('CPU'))

load_dotenv()

data_dir = os.getenv('DATA_DIR')
model_dir = os.getenv('MODEL_DIR') 


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\.venv\Lib\

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\.venv\Lib\

AttributeError: _ARRAY_API not found

AttributeError: `np.complex_` was removed in the NumPy 2.0 release. Use `np.complex128` instead.

In [ ]:
%pip uninstall numpy -y
%pip install numpy==1.24.4

Found existing installation: numpy 2.2.4
Uninstalling numpy-2.2.4:
  Successfully uninstalled numpy-2.2.4
Note: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.8 MB 8.5 MB/s eta 0:00:02
   ------- -------------------------------- 2.9/14.8 MB 9.3 MB/s eta 0:00:02
   -------- ------------------------------- 3.1/14.8 MB 9.2 MB/s eta 0:00:02
   --------- ------------------------------ 3.7/14.8 MB 5.1 MB/s eta 0:00:03
   -------------- ------------------------- 5.2/14.8 MB 5.9 MB/s eta 0:00:02
   -------------- ------------------------- 5.2/14.8 MB 5.9 MB/s eta 0:00:02
   -------------- ------------------------- 5.5/14.8 MB 3.9 MB/s eta 0:00:03
   --------------------- ------------------ 7.9/14.8 MB 4.8 MB/s eta 0:00:02
   ---------------------- ----------------- 8.4/14.8 MB 5.0 MB/s eta 0:00:02
   ---------------------- ----------------- 8.4/14.8 MB 5.0 MB/s

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.16.1 requires ml-dtypes~=0.3.1, but you have ml-dtypes 0.5.1 which is incompatible.
tensorflow-intel 2.16.1 requires tensorboard<2.17,>=2.16, but you have tensorboard 2.19.0 which is incompatible.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Paths
train_dir = r'C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\data\tf_data\cropped\train'
val_dir = r'C:\Users\hp\Downloads\BMCS 2133 Assignment\Drowning-Detection\data\tf_data\cropped\val'

# Loading the Dataset
train_data = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(128, 128),
    batch_size=32,
    label_mode='categorical'
)

val_data = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(128, 128),
    batch_size=32,
    label_mode='categorical'
)

# Model Architecture
model = models.Sequential([
    layers.Input(shape=(128, 128, 3)),
    layers.Rescaling(1./255),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')  # 3 classes: swimming, treading water, drowning
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

ModuleNotFoundError: No module named 'tensorflow.python.data.experimental.ops.iterator_model_ops'

In [ ]:
# Model Summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling_1 (Rescaling)         │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,027 (12.61 MB)

 Trainable params: 3,305,027 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Training
history = model.fit(train_data, validation_data=val_data, epochs=10)

Epoch 1/10


OverflowError: Python int too large to convert to C long

In [ ]:
# Plot Training Results
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()